In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [6]:
# Taking inputs, initializing the variables, constants

mnist_data = input_data.read_data_sets("MNIST_data/", one_hot=True)
n_train = mnist_data.train.num_examples # 55,000
n_validate = mnist_data.validation.num_examples # 5000
n_test = mnist_data.test.num_examples # 10,000

n_input = 784   # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10   # output layer (0-9 digits)

learning_rate = 1e-3
iterations = 1000
batch_size = 128

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [414]:
# Generating the tensorflow graph

ip_data = tf.placeholder("float", [None, n_input]) # 1 x 784
Y = tf.placeholder("float", [None, n_output]) # 1 x 10

# Weights, biases, z = b + Sum(wa), a = g(z) , g() = activation fn
W, b = [], []

W.append(tf.placeholder("float", [n_input, n_hidden1]))
W.append(tf.placeholder("float", [n_hidden1, n_hidden2]))
W.append(tf.placeholder("float", [n_hidden2, n_hidden3]))
W.append(tf.placeholder("float", [n_hidden3, n_output]))

b.append(tf.placeholder("float", [n_hidden1]))
b.append(tf.placeholder("float", [n_hidden2]))
b.append(tf.placeholder("float", [n_hidden3]))
b.append(tf.placeholder("float", [n_output]))

# # Activation Function used is Sigmoid

# # ------------- FEED FORWARD -----------------

z, a = [], []

z.append(tf.add(tf.matmul(ip_data, W[0]), b[0])) # 1 x 512
a.append(tf.nn.sigmoid(z[0]))
z.append(tf.add(tf.matmul(a[0], W[1]), b[1])) # 1 x 256
a.append(tf.nn.sigmoid(z[1]))
z.append(tf.add(tf.matmul(a[1], W[2]), b[2])) # 1 x 128
a.append(tf.nn.sigmoid(z[2]))
z.append(tf.matmul(a[2], W[3]) + b[3]) # 1 x 10
a.append(tf.nn.sigmoid(z[3])) # 1 x 10.... b/w (0 and 1) Activation Func = Sigmoid

layers = 4 # No of total layers including the op layer but excluding the ip layer

# # -------------- BACK PROPOGATION ------------

# # For the op layer
a_derivative = tf.multiply(a[layers - 1], tf.add(1.0, -a[layers - 1])) # f`(x) = f(x) [1 - f(x)]
Del = tf.multiply(tf.add(a[layers - 1], -Y), a_derivative) # del = (y_cap - y) (y_cap_derivative)

W[layers - 1] = W[layers - 1] - learning_rate * Del * tf.transpose(a[layers - 2]) # Updating op layer W
b[layers - 1] = b[layers - 1] - learning_rate * Del # Updating op layer biases

print b[layers - 1].shape
print W[layers - 1].shape
#     print Del.shape

for j in range(layers - 2, 0, -1): # Back propogating from the last hidden layer
    a_derivative = tf.multiply(a[j], tf.add(1.0, -a[j])) # f`(x) = f(x) [1 - f(x)]
    Del = a_derivative * (tf.matmul(Del, tf.transpose(W[j + 1]))) # Del_j = a_j` * Sum_k(Del_k * W_jk)
    W[j] = W[j] - learning_rate * Del * tf.transpose(a[j - 1]) # Updating hidden layer W
    b[j] = b[j] - learning_rate * Del # Updating hidden layer biases

# For the ip layer
a_derivative = tf.multiply(a[0], tf.add(1.0, -a[0])) # f`(x) = f(x) [1 - f(x)]
Del = a_derivative * (tf.matmul(Del, tf.transpose(W[1]))) # Del_j = a_j` * Sum_k(Del_k * W_jk)
W[0] = W[0] - learning_rate * Del * tf.transpose(ip_data) # Updating ip layer W
b[0] = b[0] - learning_rate * Del # Updating ip layer biases

# FINDING ACCURACY

correct_pred = tf.equal(tf.argmax(a[layers - 1], 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


 (?, 10)
(128, 10)


In [415]:
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    correct = 0
    Wi, bi = [], []
#     Wi.append(tf.constant(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)).eval()) # 784 x 512
#     Wi.append(tf.constant(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)).eval()) # 512 x 256
#     Wi.append(tf.constant(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)).eval()) # 256 x 128
#     Wi.append(tf.constant(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)).eval()) # 128 x 10

#     bi.append(tf.constant(tf.constant(0.1, shape=[n_hidden1])).eval()) # 512
#     bi.append(tf.constant(tf.constant(0.1, shape=[n_hidden2])).eval()) # 256
#     bi.append(tf.constant(tf.constant(0.1, shape=[n_hidden3])).eval()) # 128
#     bi.append(tf.constant(tf.constant(0.1, shape=[n_output])).eval()) # 10
    
    Wi.append(np.random.rand(n_input, n_hidden1))
    Wi.append(np.random.rand(n_hidden1, n_hidden2))
    Wi.append(np.random.rand(n_hidden2, n_hidden3))
    Wi.append(np.random.rand(n_hidden3, n_output))
    
    bi.append(np.random.rand(1, n_hidden1))
    bi.append(np.random.rand(1, n_hidden2))
    bi.append(np.random.rand(1, n_hidden3))
    bi.append(np.random.rand(1, n_output))
    
    for itr in range(iterations):
        print itr,
        batch_x, batch_y = mnist_data.train.next_batch(batch_size)
    #     sess.run([W[0], b[0]], feed_dict={ip_data: batch_x, Y: batch_y})
        for i in range(0, 1):
#             sess.run([W[0], b[0]], feed_dict={ip_data: batch_x, Y: batch_y})
            [Wi[0], bi[0], Wi[1], bi[1], Wi[2], bi[2], Wi[3], bi[3]] = sess.run([W[0], b[0], W[1], b[1], W[2], b[2], W[3], b[3]],
                     feed_dict={ip_data: batch_x, Y: batch_y,
                               W[0]: Wi[0], b[0]: bi[0],
                               W[1]: Wi[1], b[1]: bi[1],
                               W[2]: Wi[2], b[2]: bi[2],
                               W[3]: Wi[3], b[3]: bi[3]})
        print sess.run(accuracy, feed_dict={ip_data: batch_x, Y: batch_y,
                               W[0]: Wi[0], b[0]: bi[0],
                               W[1]: Wi[1], b[1]: bi[1],
                               W[2]: Wi[2], b[2]: bi[2],
                               W[3]: Wi[3], b[3]: bi[3]})
    for itr in range(iterations):
        print "Itr: " + str(itr),
        batch_x, batch_y = mnist_data.train.next_batch(batch_size)
    #     W, b, accuracy = train_with_feedforward_backpropogation(ip_data, Y, W, b, a, learning_rate, feed_dict={ip_data: batch_x, Y: batch_y})
    #     sess.run([W[0], b[0]], feed_dict={ip_data: batch_x, Y: batch_y})
#         for i in range(0, 1):
#             sess.run([W[0], b[0], W[1], b[1], W[2], b[2], W[3], b[3]], feed_dict={ip_data: batch_x, Y: batch_y})
        correct += sess.run(accuracy, feed_dict={ip_data: batch_x, Y: batch_y})
        print "Accuracy: " + str(correct / (itr + 1))

0

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_439' with dtype float and shape [512,256]
	 [[node Placeholder_439 (defined at <ipython-input-414-f07729642be4>:10) ]]

Caused by op u'Placeholder_439', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/vishal16040/.local/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/vishal16040/.local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/vishal16040/.local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/home/vishal16040/.local/lib/python2.7/site-packages/tornado/ioloop.py", line 1073, in start
    handler_func(fd_obj, events)
  File "/home/